In [1]:
import numpy as np
from scipy.signal import stft, get_window
import matplotlib.pyplot as plt
import os
import cv2
import math

In [2]:
def compute_and_extract_micro_doppler(cir_data, window_size=64, window_step=32, doppler_bins=64):
    hanning_window = get_window('hann', window_size)
    T, N = cir_data.shape
    
    spectrograms = None
    for n in range(N):
        s_n = cir_data[:, n]
        _, _, Zxx = stft(
            s_n,
            fs=1.0,
            window=hanning_window,
            nperseg=window_size,
            noverlap=window_step,
            nfft=doppler_bins,
            boundary=None,
            padded=False,
            return_onesided=False
        )
        Zxx_shifted = np.fft.fftshift(Zxx, axes=0)
        magnitude_spectrum = np.abs(Zxx_shifted)
        
        if spectrograms is None:
            num_time_frames = magnitude_spectrum.shape[1]
            spectrograms = np.zeros((doppler_bins, num_time_frames, N), dtype=np.float32)
        spectrograms[:, :, n] = magnitude_spectrum

    # Convert to dB
    spectrograms_db = 20 * np.log10(spectrograms)
    
    # Apply median filter for noise reduction
    from scipy.signal import medfilt2d
    spectrograms_filtered = np.zeros_like(spectrograms_db)
    for n in range(N):
        spectrograms_filtered[:,:,n] = medfilt2d(spectrograms_db[:,:,n], kernel_size=5)
    
    # Normalize along frequency axis
    min_vals = np.min(spectrograms_filtered, axis=0)
    max_vals = np.max(spectrograms_filtered, axis=0)
    denominator = max_vals - min_vals
    denominator[denominator == 0] = 1
    spectrograms_norm = (spectrograms_filtered - min_vals) / denominator
    
    # Apply threshold to remove weak signals
    # threshold = 0.2
    # spectrograms_norm[spectrograms_norm < threshold] = 0
    
    micro_doppler = np.sum(spectrograms_norm, axis=2)
    
    return micro_doppler, spectrograms

In [3]:

signatures_path = os.path.join("data", "b_np")
if not os.path.exists(signatures_path):
    raise FileNotFoundError(f"The directory {signatures_path} does not exist.")

output_dir = os.path.join("data", "b_sig_fig")
os.makedirs(output_dir, exist_ok=True)

signatures_output_dir = os.path.join("data", "b_sig")
os.makedirs(signatures_output_dir, exist_ok=True)

for file in os.listdir(signatures_path):
    if not file.endswith(".npy"):
        continue

    cir_data = np.load(os.path.join(signatures_path, file))

    window_size = 64
    window_step = 32
    doppler_bins = 64

    micro_doppler, spectrograms = compute_and_extract_micro_doppler(cir_data, window_size, window_step, doppler_bins)

    # Save the aggregated spectrogram as .npy file
    sig_output_filename = os.path.join(signatures_output_dir, file)
    np.save(sig_output_filename, micro_doppler)

    # Resize the spectrogram to 256x256
    resized_image = cv2.resize(micro_doppler, (256, 256), interpolation=cv2.INTER_LINEAR)
    
    # Save the image
    output_filename = file.replace('.npy', '.png')
    output_path = os.path.join(output_dir, output_filename)
    plt.imsave(output_path, resized_image, cmap='gray', format='png')